In [ ]:
from selenium import *

In [1]:
from selenium import webdriver

In [2]:
from selenium.webdriver.common.keys import Keys

In [3]:
from selenium.common.exceptions import NoSuchAttributeException

In [4]:
from time import sleep

In [5]:
from selenium.webdriver.common.by import By

In [6]:
# On crée une instance du web-driver Firefox et on va sur la page de données financières :
driver = webdriver.Firefox()
driver.get("https://investir.lesechos.fr/actions/consensus/consensus-donnees-financieres.html")
# Il y a des alternatives pour les autres navigateurs :
# webdriver.Chrome()

sleep(10) # Selon la qualité de connexion et les perfs de la machine, temps d'attente nécessaire.

In [7]:
donnees = {}

driver.get("https://investir.lesechos.fr/actions/consensus/consensus-donnees-financieres.html")

sleep(2) #sleep pour être sur que la fenêtre des cookies apparaît
try : # S'il faut valider les cookies :
    boutton_cookies = driver.find_element_by_id("didomi-notice-agree-button")
    boutton_cookies.click() # accepter les cookies
except NoSuchElementException : # Si pas besoin de valider les cookies :
    pass
    
is_last_page = False
while not is_last_page:

    # On récupère une liste d'éléments correspondant à chaque ligne du tableau 
    tableau = driver.find_elements_by_class_name("tr-clicable")
    
    for l in tableau:
        valeurs = []
        valeurs = l.find_elements_by_class_name("dt-body-right")
        
        # Si le nom est trop long et qu'il y a une version complète :
        if l.find_element_by_css_selector("a").get_attribute("title") != "": 
            nom = l.find_element_by_css_selector("a").get_attribute("title")
        # Si le nom est suffisamment court pour être affiché tel quel sur la page :
        else :
            nom = l.find_element_by_css_selector("a").text
            
        # On remplit notre dictionnaire pour la clé de l'entreprise en cours de traitement :
        donnees[nom] = [int(val.text.replace(" ","")) for val in valeurs] # retrait des espaces et passage en int
        print(nom, donnees[nom])
        
    # On récupère l'élément de changement de page.
    next_page = driver.find_element_by_id("tableConsensusDonneesFinanciers_next")

    # Si pas de next page possible, le flag de sortie du while est activé :
    if next_page.get_attribute("class") == "paginate_button next disabled":
        is_last_page = True
        
    # Sinon, on clique sur l'élément pour aller à la page suivante et continuer le scrapping.
    else:
        tableau = [] # On vide le tableau
        print("--------------",next_page.get_attribute("class"),"--------------")
        next_page.click()
        
    sleep(2) # Temps d'attente pour donner le temps au pc/navigateur.

ACCOR [4049, 825, 497, 1621, -391, -665, 2436, 110, -161]
ADP [4700, 1772, 1062, 2137, 168, -744, 2793, 545, -247]
AIR FRANCE -KLM [27189, 4128, 1141, 11088, -1689, -4548, 14650, 593, -2182]
AIR LIQUIDE [21920, 5931, 3794, 20485, 5928, 3790, 21359, 6132, 4033]
AIRBUS [70478, 9849, 6946, 49912, 4513, 1706, 51392, 5284, 2631]
ALSTOM [8072, 690, 570, 8201, 838, 559, 7735, 808, 535]
AMUNDI [2707, 1331, 1330, 2595, 1312, 1254, 2796, 1394, 1373]
ARKEMA [8738, 1457, 926, 7884, 1182, 619, 7979, 1209, 670]
ATOS [11588, 1554, 1190, 11181, 1484, 1002, 11361, 1612, 1073]
AXA [103532, 7608, 7608, 96723, 5779, 5779, 98635, 8329, 8403]
BIOMERIEUX [2675, 576, 389, 3118, 807, 613, 3252, 843, 600]
BNP PARIBAS [44597, 0, 13260, 44275, 0, 14081, 44461, 0, 14270]
BOUYGUES [37929, 3548, 1677, 34694, 3233, 1124, 36730, 3528, 1613]
BUREAU VERITAS [5100, 1057, 832, 4601, 819, 615, 4800, 969, 720]
CAPGEMINI [14125, 2105, 1741, 15848, 2458, 1879, 16880, 2646, 1985]
CARREFOUR [72450, 4417, 2080, 70719, 4465, 2173

In [20]:
from csv import writer

def append_list_as_row(nom_csv, liste):
    # On ouvre le csv en mode append :
    with open(nom_csv, 'a+', newline='') as csvfile:
        # Create a writer object from csv module
        csv_writer = writer(csvfile)
        # Add contents of list as last row in the csv file
        csv_writer.writerow(liste)

In [21]:
for key,value in donnees.items():
    print(key)
    print(value[0:3])
    print(value[3:6])
    print(value[6:])

    append_list_as_row("donnees_financieres.csv", [key]+[str(val) for val in value[0:3]]+["2019"])
    append_list_as_row("donnees_financieres.csv", [key]+[str(val) for val in value[3:6]]+["2020"])
    append_list_as_row("donnees_financieres.csv", [key]+[str(val) for val in value[6:]]+["2021"])

ACCOR
[4049, 825, 497]
[1621, -391, -665]
[2436, 110, -161]
ADP
[4700, 1772, 1062]
[2137, 168, -744]
[2793, 545, -247]
AIR FRANCE -KLM
[27189, 4128, 1141]
[11088, -1689, -4548]
[14650, 593, -2182]
AIR LIQUIDE
[21920, 5931, 3794]
[20485, 5928, 3790]
[21359, 6132, 4033]
AIRBUS
[70478, 9849, 6946]
[49912, 4513, 1706]
[51392, 5284, 2631]
ALSTOM
[8072, 690, 570]
[8201, 838, 559]
[7735, 808, 535]
AMUNDI
[2707, 1331, 1330]
[2595, 1312, 1254]
[2796, 1394, 1373]
ARKEMA
[8738, 1457, 926]
[7884, 1182, 619]
[7979, 1209, 670]
ATOS
[11588, 1554, 1190]
[11181, 1484, 1002]
[11361, 1612, 1073]
AXA
[103532, 7608, 7608]
[96723, 5779, 5779]
[98635, 8329, 8403]
BIOMERIEUX
[2675, 576, 389]
[3118, 807, 613]
[3252, 843, 600]
BNP PARIBAS
[44597, 0, 13260]
[44275, 0, 14081]
[44461, 0, 14270]
BOUYGUES
[37929, 3548, 1677]
[34694, 3233, 1124]
[36730, 3528, 1613]
BUREAU VERITAS
[5100, 1057, 832]
[4601, 819, 615]
[4800, 969, 720]
CAPGEMINI
[14125, 2105, 1741]
[15848, 2458, 1879]
[16880, 2646, 1985]
CARREFOUR
[72450,